In [1]:
# import the needed modules
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as req
import numpy as np
from IPython.display import HTML

In [2]:
#request with Requests library, using Postman interceptor - empty IS-Academia form
base_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247'
r = req.get(base_url)

In [3]:
# define BeautifulSoup object to parse the html doc
soup = bs(r.text,'lxml')

# Parsing the HTML form

In the following the parameters model, section, academic year, academic cycle and season are parsed, so that each variable is linked to its corresponding numeric code through a dictionary. Thereafter, the parameters names are obtained as the IS_Academia form accepts them.

In [4]:
# parse the model code
parse_model = soup.find_all('input')
model = {}
html_model = 'ww_i_reportmodel'
xls_model = 'ww_i_reportModelXsl'
for ind in parse_model:
    if ind['name']==html_model:
        model[html_model]=ind['value']
    if ind['name']==xls_model:
        model[xls_model]=ind['value']
model

{'ww_i_reportModelXsl': '133685271', 'ww_i_reportmodel': '133685247'}

In [5]:
# parse the section Informatique
section = {'Informatique' : soup.find('option',string='Informatique')['value']}
section

{'Informatique': '249847'}

In [6]:
# parse the academic year - loop over the years and find the code corresponding to the year string
academic_year = {}
for year in range(2007,2017):
    period = str(year)+'-'+str(year+1)
    academic_year[period] = soup.find('option', string = period)['value']
academic_year

{'2007-2008': '978181',
 '2008-2009': '978187',
 '2009-2010': '978195',
 '2010-2011': '39486325',
 '2011-2012': '123455150',
 '2012-2013': '123456101',
 '2013-2014': '213637754',
 '2014-2015': '213637922',
 '2015-2016': '213638028',
 '2016-2017': '355925344'}

In [7]:
# parse the academic cycle - define the sought cycles and look for the corresponding codes, as above
bachelor = []
master = []
for j in range(1,7):
    bachelor.append('Bachelor semestre '+str(j))
for j in range(1,5):
    master.append('Master semestre '+str(j))
bachelor_master = bachelor+master

# add the master projects
bachelor_master.append('Projet Master automne')
bachelor_master.append('Projet Master printemps')

academic_cycle = {}
for cycle in bachelor_master:
    academic_cycle[cycle] = soup.find('option', string = cycle)['value']
academic_cycle

{'Bachelor semestre 1': '249108',
 'Bachelor semestre 2': '249114',
 'Bachelor semestre 3': '942155',
 'Bachelor semestre 4': '942163',
 'Bachelor semestre 5': '942120',
 'Bachelor semestre 6': '942175',
 'Master semestre 1': '2230106',
 'Master semestre 2': '942192',
 'Master semestre 3': '2230128',
 'Master semestre 4': '2230140',
 'Projet Master automne': '249127',
 'Projet Master printemps': '3781783'}

In [8]:
# parse the academic season - same procedure as above (actually not necessary)
season = {}
season["Semestre d'automne"]=soup.find('option', string = "Semestre d'automne")['value']
season['Semestre de printemps']=soup.find('option', string = 'Semestre de printemps')['value']
season

{"Semestre d'automne": '2936286', 'Semestre de printemps': '2936295'}

In [9]:
# get the variables' names to fill in the IS-Academia form
variables = []
args = soup.find_all('select')
for var in args:
    variables.append(var['name'])
variables

['ww_x_UNITE_ACAD',
 'ww_x_PERIODE_ACAD',
 'ww_x_PERIODE_PEDAGO',
 'ww_x_HIVERETE']

# Extraction of the GPS identifier

The crucial information to be retrieved is now the parameter 'ww_x_GPS', which identifies the table to be downloaded.

We define a suitable function to "hack" the value of the variable ww_x_GPS. Let us consider the following example:

class="ww_x_GPS" href="javascript:void(0)" onclick="loadReport('ww_x_GPS=1378362120');return false;">Informatique, 2012-2013, Bachelor semestre 2

The GPS code is reported within the call to the html function 'loadReport', which is called when clicking the dataset. The function captureGPS seeks for the equal sign '=' inside the round parentheses and stops before the apostrophe sign. Once got the value of ww_x_GPS, one can use it to perform a request to the IS-Academia server passing a proper base url.

In [10]:
# function to detect the GPS code of the table
# strong assumption of the string's structure!
def captureGPS(string):
    letter = 0
    while(string[letter]!='='):
        letter=letter+1 #advance up to the =
    letter = letter+1 # reach the first number of the GPS code
    GPS = string[letter]
    letter = letter+1
    while (string[letter]!="'"):
        GPS = GPS+string[letter]
        letter = letter+1
    return GPS

GPS = 'ww_x_GPS'

# Import the tables from IS-Academia

We begin now the importation of all the tables from the server. We define lists containing the values of the required parameters (section, year, cycle) and then seek for the ww_x_GPS parameter of each single table. We thus define a dataframe object and append progressively the datasets with the methods provided by the Pandas library. The index of the resulting dataframe will be a combination of name and Sciper number, which is of course not unique.

In [11]:
# define the parameters to perform the requests
year = []
for j in range(2007,2017):
    year.append(str(j)+'-'+str(j+1))

# the periods list has already been defined during the parsing operations
#bachelor_master

course = 'Informatique'

# model variables have already been defined
# html_model, xls_model

dataset = pd.DataFrame(); #empty dataframe where to append all the tables

In [37]:
# for now, only one year
request_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?'
base_url = 'http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_i_reportModel=133685247'

for cur_year in year:
    for cycle in bachelor_master: # loop over the bachelor and master cycles
        # define a search dictionary using the previous variables
        research_keys = {}
        research_keys[variables[0]] = section[course]
        research_keys[variables[1]] = academic_year[cur_year]
        research_keys[variables[2]] = academic_cycle[cycle]
        research_keys['ww_b_list'] = 1
        research_keys['ww_x_HIVERETE'] = 'null' # not important to extract the season - linked to the cycle
        research_keys[html_model] = model[html_model]
        research_keys[xls_model] = model[xls_model]
        
        r0 = req.get(base_url, params = research_keys)
        s0 = bs(r0.text,'lxml') # beautifulsoup object from where to extract the GPS parameter
        
        # check for correctness of the call to captureGPS - missing data might occur
        if(len(s0.find_all('a')) > 0):
            research_keys[GPS]=captureGPS(s0.find_all('a')[1]['onclick'])
            r = req.get(request_url, params = research_keys)
            s = bs(r.text,'lxml')
    
            # s is the beautifulsoup object pointing to the table where are interested in
            # define the columns names
            attributes_list = s.find_all('th')[2:]
            attributes = []
            for ind in attributes_list:
                attributes.append(ind.contents[0])
            
            # import the dataframe and manipulate properly the layout
            df = pd.read_html(r.url)
            # ensure the dataframe is not empty, otherwise skip
            if(df[0].shape[0]>1 & df[0].shape[1]>1):
                df = df[0][3:] # data is a list of dataframes of length 1; remove the first 2 rows ad the last 2 empty columns
                del df[11]
                del df[12]
                df.columns = attributes
                df = df.set_index('No Sciper')
            
                # add year and cycle information
                df['Période académique']=cycle
                df['Année académique']=cur_year
    
                # append dataframe
                dataset = dataset.append(df)
        
    # useful for the user to understand the underlying process
    message = 'year: '+str(cur_year)+' parsed'
    print(message)
    
print('Finished parsing')

year: 2007-2008 parsed
year: 2008-2009 parsed
year: 2009-2010 parsed
year: 2010-2011 parsed
year: 2011-2012 parsed
year: 2012-2013 parsed
year: 2013-2014 parsed
year: 2014-2015 parsed
year: 2015-2016 parsed
year: 2016-2017 parsed
Finished parsing


In [39]:
# save dataframe to load it in another ipython notebook, for sake of clarity
dataset.to_excel('dataISAcademia.xls',sheet_name='Sheet1')